---

1. Reads a static CSV file containing wind power data (with columns such as Date/Time, LV ActivePower (kW), etc.) using Spark.
2. Assigns a sequential index (row_id) to each row in the CSV.
3. Uses a rate streaming source to emit rows at a controlled rate.
4. Joins the streaming source with the static DataFrame on row_id.
5. Publishes the resulting joined rows to a Kafka topic (xenon-topic).

---

In [1]:
from pyspark.sql import SparkSession as ss, Row as r
from pyspark.sql.functions import to_json, struct, col, lit
from pyspark.sql.types import LongType as lt

Initialize Spark Session:
- Name the application `KafkaProducer`
- Connect to the Spark master at `spark://spark-master:7077`
- Include the spark-sql-kafka package so Spark can write to Kafka

In [20]:
sprk = ss.builder \
    .appName("KafkaProducer") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.5") \
    .config("spark.jars.repositories", "https://repos.spark-packages.org") \
    .getOrCreate()

File path (inside the container) for the CSV dataset

In [21]:
fp = "/data/dataset.csv"

Load the static CSV DataFrame with inferred schema  
`header=True` means first row in CSV is treated as column names

In [22]:
df = sprk.read.csv(fp, header=True, inferSchema=True).cache()

In [23]:
df.show(5)

+----------------+-------------------+----------------+-----------------------------+------------------+
|       Date/Time|LV ActivePower (kW)|Wind Speed (m/s)|Theoretical_Power_Curve (KWh)|Wind Direction (°)|
+----------------+-------------------+----------------+-----------------------------+------------------+
|01 01 2018 00:00|   380.047790527343|5.31133604049682|             416.328907824861|  259.994903564453|
|01 01 2018 00:10|    453.76919555664|5.67216682434082|             519.917511061494|   268.64111328125|
|01 01 2018 00:20|   306.376586914062|5.21603679656982|             390.900015810951|  272.564788818359|
|01 01 2018 00:30|   419.645904541015|5.65967416763305|             516.127568975674|  271.258087158203|
|01 01 2018 00:40|   380.650695800781|5.57794094085693|             491.702971953588|  265.674285888671|
+----------------+-------------------+----------------+-----------------------------+------------------+
only showing top 5 rows



Function to add an index (row_id) to each row in the DataFrame

In [41]:
def add_idx_row(row, index):
    row_dict = row.asDict() # Convert the Row to a dict, add "row_id", then rebuild a Row
    row_dict["row_id"] = index
    return r(**row_dict)

Convert the DataFrame to an RDD, zip each row with a unique index,  
then map them back to a new row object that includes row_id

In [42]:
rdd = df.rdd.zipWithIndex().map(lambda x: add_idx_row(x[0], x[1]))

Create a new schema based on the original schema plus row_id as a LongType

In [43]:
new_schema = df.schema.add("row_id", lt())

Rebuild a Spark DataFrame with the new schema (including row_id)

In [44]:
df_w_idx = sprk.createDataFrame(rdd, schema=new_schema)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonUtils.getBroadcastThreshold.
: java.lang.NullPointerException: Cannot invoke "org.apache.spark.api.java.JavaSparkContext.sc()" because "jsc" is null
	at org.apache.spark.api.java.JavaSparkContext$.toSparkContext(JavaSparkContext.scala:794)
	at org.apache.spark.api.python.PythonUtils$.getBroadcastThreshold(PythonUtils.scala:87)
	at org.apache.spark.api.python.PythonUtils.getBroadcastThreshold(PythonUtils.scala)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:569)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Ga

In [26]:
df_w_idx.printSchema()

root
 |-- Date/Time: string (nullable = true)
 |-- LV ActivePower (kW): double (nullable = true)
 |-- Wind Speed (m/s): double (nullable = true)
 |-- Theoretical_Power_Curve (KWh): double (nullable = true)
 |-- Wind Direction (°): double (nullable = true)
 |-- row_id: long (nullable = true)



Create a streaming DataFrame that generates data at a fixed rate.  
Here, `rowsPerSecond=10` means `10` rows are produced every second.

In [27]:
new_df = sprk.readStream \
    .format("rate") \
    .option("rowsPerSecond", 10) \
    .load()

Rename the automatic `value` column (from `rate` source) to `row_id`

In [46]:
new_df = new_df.withColumnRenamed("value", "row_id")

Count how many rows are in the static CSV DataFrame

In [29]:
c = df_w_idx.count()

In [31]:
print(c)

50530


Filter the streaming DataFrame so that it emits only row_ids < total row count  
This ensures we don't exceed the number of rows in the CSV

In [32]:
new_df = new_df.filter(col("row_id") < lit(c))

Join the streaming DataFrame with the static DataFrame on row_id  
`timestamp` is auto-generated by the rate source, so drop it

In [47]:
joined_df = new_df.join(df_w_idx, on="row_id", how="left_outer").drop("timestamp")

In [34]:
joined_df.printSchema()

root
 |-- row_id: long (nullable = true)
 |-- Date/Time: string (nullable = true)
 |-- LV ActivePower (kW): double (nullable = true)
 |-- Wind Speed (m/s): double (nullable = true)
 |-- Theoretical_Power_Curve (KWh): double (nullable = true)
 |-- Wind Direction (°): double (nullable = true)



Convert the joined DataFrame to JSON: we select all columns and nest them in a JSON object named "value"

In [35]:
out = joined_df.select(
    to_json(struct([col(x) for x in joined_df.columns])).alias("value")
)

In [36]:
out.printSchema()

root
 |-- value: string (nullable = true)



Write the JSON objects to Kafka:
- `kafka.bootstrap.servers` must be set to the Kafka service inside Docker
- `checkpointLocation` is where Spark stores offsets and progress
- `trigger(once=True)` runs the stream once and then terminates

In [38]:
q = out.writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("topic", "xenon-topic") \
    .option("checkpointLocation", "/tmp/kafka_checkpoint") \
    .outputMode("append") \
    .trigger(once=True) \
    .start()

Wait for streaming to finish

In [48]:
q.awaitTermination()

Stop the Spark session

In [40]:
sprk.stop()